In [6]:
import cv2
import os
import imutils

personName = 'Eliu'
dataPath = '/content/drive/MyDrive/'
personPath = dataPath + '/' + personName

if not os.path.exists(personPath):
    print('Carpeta creada: ',personPath)
    os.makedirs(personPath)
    
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cap = cv2.VideoCapture('video/miVideo.mp4')
print('detectando')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

print('YA detectando')
count = 0

while True:

    ret, frame = cap.read()
    if ret == False: break
    frame =  imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = frame.copy()
                                      
    faces = faceClassif.detectMultiScale(gray,1.2,5)   #(image, scaleFactor, minNeighbors)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        rostro = auxFrame[y:y+h,x:x+w]
        rostro = cv2.resize(rostro,(150,150),interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(personPath + '/rostro_{}.jpg'.format(count),rostro)
        count = count + 1
    cv2.imshow('frame',frame)
    k =  cv2.waitKey(1)
    if k == 27 or count >= 350:
        break
print('FIN')

cap.release()
cv2.destroyAllWindows()

detectando
YA detectando
FIN


In [12]:
import cv2
import os

imagesPath = "rostros/" # Cambia a la ruta donde hayas almacenado la carpeta con las imágenes
imagesPathList = os.listdir(imagesPath)

if not os.path.exists('rostros1/'):
    print('Carpeta creada: Rostros1')
    os.makedirs('rostros1/')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

count = 0
for imageName in imagesPathList:
    image = cv2.imread(imagesPath+'/'+imageName)
    imageAux = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = faceClassif.detectMultiScale(gray, 1.2, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(image, (x,y),(x+w,y+h),(128,0,255),2)
    cv2.rectangle(image,(10,5),(450,25),(255,255,255),-1)

    for (x,y,w,h) in faces:
            rostro = imageAux[y:y+h,x:x+w]
            rostro = cv2.resize(rostro,(150,150), interpolation=cv2.INTER_CUBIC)
            #cv2.imshow('rostro',rostro)
            cv2.waitKey(0)
            cv2.imwrite('rostros1/rostro_{}.jpg'.format(count),rostro)
            count = count +1

cv2.destroyAllWindows()

### XD


In [14]:
from skimage.io import imread_collection , concatenate_images
import numpy as np
import matplotlib.pyplot as plt
import copy  as cp
import copy

# path 
folderEliu = 'Eliu/*.jpg'
folderOthers = 'rostros1/*.jpg'

# load data 
imagesEliu = imread_collection(folderEliu)
imagesOthers = imread_collection(folderOthers) 

# len
nEliu  = len(imagesEliu)
nOthers = len(imagesOthers)

#HACER RANDOM Y PARA NO SESGASR EL MODELO 
#AUMENTAR DATOS A LA CLASE QUE FALTE PARA TENER IGUAL CANTIDAD Y NO DESBALANCEAR EL MODELO 

# union data
images = np.append(imagesEliu, imagesOthers, axis=0)

print("Total de imagenes: ",len(images))

Total de imagenes:  392


In [15]:
def Create_Y():
     return [0]*nEliu + [1]*nOthers
Y = Create_Y()

Y = np.array(Y)
X = np.array(images)

In [16]:
from skimage.transform import resize
X=resize(X,(len(images),64,64,3))

In [18]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

modelo=Sequential() #Varias capas apiladas entre ellas

modelo.add(Conv2D(200,(3,3),input_shape=X.shape[1:]))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
#La primera capa de CNN seguida por las capas de Relu y MaxPooling

modelo.add(Conv2D(100,(3,3)))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
#La segunda capa de convolución seguida por las capas de Relu y MaxPooling

modelo.add(Conv2D(50,(3,3)))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))
#La tercera capa de convolución seguida por las capas de Relu y MaxPooling

modelo.add(Flatten()) #Imagen profunda la vamos a hacer plana, es decir solo una dimension,
#va tener toda nuestra informacion de la cnn

modelo.add(Dropout(0.5)) # Apagamos 50% de las neuronas cada paso, asi evitamos sobreajustar
#(evitar un solo camino de entrenamiento)

modelo.add(Dense(50,activation='relu'))
#Capa Densa de 50 neuronas

modelo.add(Dense(2,activation='softmax')) #La capa final softmax con dos salidas para dos categorías
#Softmax nos indica que tanta probabilidad tiene cada clase, y por ende saber cual tiene la maxima probabilidad

modelo.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Ya que tenemos 2 clases debemos esperar que shape[1] de y_train,y_val y y_test cambie de 0 a 1
Y=to_categorical(Y)

Y[0]

array([1., 0.], dtype=float32)

In [20]:
Y[len(Y)-1]

array([0., 1.], dtype=float32)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0)

In [22]:
checkpoint = ModelCheckpoint('CNN/model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [ ]:
history=modelo.fit(X_train,Y_train,epochs=20,callbacks=[checkpoint],validation_split=0.2) #,shuffle = True)

In [24]:
print(modelo.evaluate(X_test,Y_test))

4/4 [==============================] - 0s 58ms/step - loss: 0.0085 - accuracy: 1.0000
[0.008468396961688995, 1.0]


In [25]:
#Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#Print Confusion matrix
    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

np.set_printoptions(precision=2)

In [29]:

#Making prediction
#train_data,test_data,train_target,test_target

y_pred=modelo.predict(X_test) 
y_true=np.argmax(Y_test,axis=1)

""""
#Plotting the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_mtx=confusion_matrix(y_true,y_pred)

class_names=['eliu', 'Desconocido']
# Plotting non-normalized confusion matrix
plot_confusion_matrix(y_true, y_pred, classes = class_names,title = 'Confusion matrix, without normalization')

"""

4/4 [==============================] - 0s 60ms/step


'"\n#Plotting the confusion matrix\nfrom sklearn.metrics import confusion_matrix\nconfusion_mtx=confusion_matrix(y_true,y_pred)\n\nclass_names=[\'eliu\', \'Desconocido\']\n# Plotting non-normalized confusion matrix\nplot_confusion_matrix(y_true, y_pred, classes = class_names,title = \'Confusion matrix, without normalization\')\n\n'

### Pruebas


In [1]:

from keras.models import load_model
import cv2
import numpy as np

In [2]:
modelo = load_model('CNN/model-018.model')



In [ ]:
from skimage.transform import resize
import matplotlib.pyplot as plt
face_clsfr = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

source=cv2.VideoCapture('VID_20201206_001148.mp4')

labels_dict={0:'eliu',1:'Desconocido'}
color_dict={0:(0,255,0),1:(0,0,255)}

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=resize(face_img,(len(face_img),64,64,3))
    
        result=modelo.predict(resized)
        label=np.argmax(result,axis=1)[0]
        
        cv2.putText(img,'{}'.format(label),(x,y-50),1,1.3,(86,155,35),1,cv2.LINE_AA)
    
        if label == 0: 
               # print(result[0][1])
                cv2.putText(img,'{}'.format(labels_dict[label]),(x,y-25),2,1.1,(255,255,0),1,cv2.LINE_AA)
                cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,0),2)
        #elif label == 1:
        else:
                cv2.putText(img,'{}'.format(labels_dict[label]),(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
                cv2.rectangle(img, (x,y),(x+w,y+h),(0,0,255),2)    
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==1):
        break
        
cv2.destroyAllWindows()
source.release()